<a href="https://colab.research.google.com/github/TharunVarma1/AIML/blob/main/Generative_AI_2265.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Feature Selection Approach Combining Genetic Algorithm and Recursive Feature Elimination for Solar Radiation Estimation

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor


# **Reading the Dataset**

In [2]:
df=pd.read_excel('/content/Data_Solar Radiation.xlsx')
df

,Unnamed: 0,Unnamed: 1,Input Variable,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Target Variable
0,From Date,To Date,PM2.5,PM10,NO,NO2,NOx,NH3,SO2,CO,...,Benzene,Toluene,Temp,RH,WS,WD,BP,Xylene,AT,SR
1,01-01-2024 00:00,02-01-2024 00:00,65.1,107.53,2.37,19.85,12.48,10.43,9.05,0.61,...,0.61,2.19,26.42,67.65,0.43,187.48,710.71,1.69,18.77,87.73
2,02-01-2024 00:00,03-01-2024 00:00,74.23,113.42,2.45,21.28,13.27,10.18,8.89,0.64,...,0.6,2.2,26.41,71.19,0.52,172.35,710.31,1.69,18.3,82.34
3,03-01-2024 00:00,04-01-2024 00:00,81.83,123.53,2.59,22.27,13.92,9.86,11.25,0.69,...,0.61,2.19,26.66,70.97,0.76,173.54,709.95,1.68,18.23,84.22
4,04-01-2024 00:00,05-01-2024 00:00,66.04,121.96,2.44,25.01,14.56,10.09,10.85,0.69,...,0.61,2.21,26.41,71.06,0.73,172.96,709.91,1.69,19.21,81.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,15-03-2025 00:00,16-03-2025 00:00,30.62,112.89,14.12,30.21,22.96,19.29,6.03,0.72,...,0.81,2.93,29.03,47.23,1.03,274.81,712.88,2.25,35.31,108.25
439,16-03-2025 00:00,17-03-2025 00:00,28.25,95.54,13.85,29.64,22.02,18.92,5.88,1.01,...,0.8,2.9,29.19,41.73,1.2,296.65,712.8,2.23,35.44,105.14
440,17-03-2025 00:00,18-03-2025 00:00,21.31,91.82,14,29.94,22.51,19.11,5.93,3.18,...,0.81,2.95,28.72,39.75,1.02,305.13,712.54,2.26,35.22,110.59
441,18-03-2025 00:00,19-03-2025 00:00,34.51,110.69,13.27,28.4,21.57,18.12,5.67,0.77,...,0.82,2.97,29.43,51.17,0.63,258.93,712.49,2.28,35,110.24


# **Finding the Missing Values**

In [3]:
num_rows = df.shape[0]
num_columns = df.shape[1]
print(df)
missing_values = df.isnull().sum()
print("\nMissing values per column:")
print(missing_values)



           Unnamed: 0        Unnamed: 1 Input Variable Unnamed: 3 Unnamed: 4  \
0           From Date           To Date          PM2.5       PM10         NO   
1    01-01-2024 00:00  02-01-2024 00:00           65.1     107.53       2.37   
2    02-01-2024 00:00  03-01-2024 00:00          74.23     113.42       2.45   
3    03-01-2024 00:00  04-01-2024 00:00          81.83     123.53       2.59   
4    04-01-2024 00:00  05-01-2024 00:00          66.04     121.96       2.44   
..                ...               ...            ...        ...        ...   
438  15-03-2025 00:00  16-03-2025 00:00          30.62     112.89      14.12   
439  16-03-2025 00:00  17-03-2025 00:00          28.25      95.54      13.85   
440  17-03-2025 00:00  18-03-2025 00:00          21.31      91.82         14   
441  18-03-2025 00:00  19-03-2025 00:00          34.51     110.69      13.27   
442  19-03-2025 00:00  19-03-2025 09:24           36.5       82.5       1.49   

    Unnamed: 5 Unnamed: 6 Unnamed: 7 Un

In [4]:
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)
X=df.iloc[:, 2:-1]
Y=df.iloc[:, -1]
X.fillna(X.mean(), inplace=True)
Y.fillna(Y.mean(), inplace=True)

<ipython-input-4-d42a25e75e87>:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X.fillna(X.mean(), inplace=True)
<ipython-input-4-d42a25e75e87>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)
<ipython-input-4-d42a25e75e87>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.mean(), inplace=True)
<ipython-i

# **Test and Training the Cases**

In [15]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# Scale the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Convert y_train and y_test to NumPy arrays
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)
# Scale the target variable
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)
# Feature selection using Recursive Feature Elimination (RFE)
estimator = RandomForestRegressor(random_state=42)
selector = RFE(estimator, n_features_to_select=5, step=1) # Select top 5 features
selector = selector.fit(X_train_scaled, y_train_scaled.ravel()) #Fit on scaled data

# **Selecting the features**

In [13]:
# Get the selected features
selected_features = X.columns[selector.support_]
print("Selected Features:", selected_features)

# Transform the datasets using the selected features
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)

Selected Features: Index(['NO2', 'RH', 'WD', 'Xylene', 'AT'], dtype='object', name=0)


# **Finding the Result**

In [14]:
# Now you can use X_train_selected and X_test_selected with your model
# Example using a simple neural network:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_selected.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1)) # Output layer for regression

model.compile(optimizer='adam', loss='mse')
model.fit(X_train_selected, y_train_scaled, epochs=50, batch_size=32, verbose=0) #Train the model

y_pred_scaled = model.predict(X_test_selected)
y_pred = scaler_y.inverse_transform(y_pred_scaled)


from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Mean Squared Error: 421.9945181537548
R-squared: 0.4568419117279804
